In [1]:
# do not need
# wandb.init(reinit=True)

In [2]:
import random
import pandas as pd 
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
import matplotlib.pyplot as plt
from PIL import Image
import os
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications import VGG16
import tensorflow.keras.applications as tfa
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow import keras
tf.random.set_seed(42)

import wandb
from wandb.keras import WandbCallback
tf.keras.backend.clear_session()
# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

In [3]:
from keras import backend as K

In [4]:
dupicate_removed = pd.read_csv('../input/duplicate-removed-train/duplicate_removed_train.csv')

# Havent used:
1. activation -> create_model
2. number of neurons in dense layer -> create_model
3. loss function  -> compile_model
4. data augmentation ->load_train

In [5]:
#meta = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
meta = pd.read_csv('../input/duplicate-removed-train/duplicate_removed_train.csv')

In [6]:
def append_ext(fn):
    return fn+".jpg"

In [7]:
labels = meta[['Id','Pawpularity']]
labels['Pawpularity_norm'] = labels['Pawpularity']/100
labels["Id"] = labels["Id"].apply(append_ext)
labels.to_csv('labels.csv',encoding='utf-8', index=False)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
def load_train(path, batch_size):
        # path /kaggle/input/petfinder-pawpularity-score/
    labels = pd.read_csv('./labels.csv')
       
    train_datagen = ImageDataGenerator(validation_split = 0.25, 
                                   #rescale = 1/255.
                                   #vertical_flip = True, 
                                   horizontal_flip = True
                                   #rotation_range = 45,
                                   #width_shift_range=0.05,
                                   #height_shift_range=0.05
                                      )
    
    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory= path +'train/',
        x_col='Id',
        y_col='Pawpularity_norm',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='raw',
        subset='training',
        seed=42)

    return train_gen_flow

In [9]:
def load_test(path, batch_size):
    labels = pd.read_csv('./labels.csv')
    test_datagen=ImageDataGenerator(
        validation_split=0.25,
        horizontal_flip = True
        #rescale=1./255
    )
    
# flow_from_dataframe -> Takes the dataframe and the path to a directory + generates batches.
# The generated batches contain augmented/normalized data.
    test_gen_flow=test_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'train/',
        x_col='Id',
        y_col='Pawpularity_norm',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='raw',
        subset='validation',
        seed=42)

    return test_gen_flow

In [10]:
def create_model(input_shape, model_name, head, dropout):
    
    print('create_model() started...')
    
    inputs = keras.Input(shape=input_shape)
    x = inputs
  
    if model_name == 'ResNet50':
        
        print('loading pretrained model ResNet50...')
         
        backbone = ResNet50(weights='imagenet', 
                        input_shape=input_shape,
                        include_top=False)
        
          # preprocess_input is about:
          # The images are converted from RGB to BGR, then each color channel is zero-centered with respect to the ImageNet dataset, without scaling.
        print('create_model() pictures mormalization ')
        x = tf.keras.applications.resnet50.preprocess_input(x)
        
    elif model_name == 'VGG16':
        
        print('loading pretrained model VGG16...')
        backbone = VGG16(weights='imagenet', 
                        input_shape=input_shape,
                        include_top=False)
        print('create_model() pictures mormalization ')
        x = tf.keras.applications.vgg16.preprocess_input(x)
        
    print('create_model() pictures mormalization done...')
    
    x = backbone(x, training=True)

    set_trainable = False
    
#     for layer in backbone.layers:
#         if model_name == 'ResNet50':
#             fine_tune_from = 'conv3'
#             if fine_tune_from in layer.name :
#                 set_trainable = True
#             if set_trainable:
#                 layer.trainable = True
#             else:
#                 layer.trainable = False
                
#         elif model_name == 'VGG16':
#             fine_tune_from = 'block4_conv1'
#             if fine_tune_from in layer.name :
#                 set_trainable = True
#             if set_trainable:
#                 layer.trainable = True
#             else:
#                 layer.trainable = False

    if head == 'GPA_sigmoid':
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        output = keras.layers.Dense(1, activation='sigmoid')(x)
        
    
    elif head == 'GPA_FC_sigmoid':
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dense(3, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        output = keras.layers.Dense(1, activation='sigmoid')(x)
        
    elif head == 'MP_sigmoid':
        x =  tf.keras.layers.MaxPooling2D()(x)
        output = keras.layers.Dense(1, activation='sigmoid')(x)
    
    elif head == 'GPA_logits':
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        output = keras.layers.Dense(1)(x)
        
    elif head == 'Flatten_FC_sigmoid_DO':
    
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(dropout)(x)
        output = keras.layers.Dense(1,activation='sigmoid')(x)
        
    elif head == 'Flatten_FC_relu_DO':
    
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(dropout)(x)
        output = keras.layers.Dense(1)(x)
    
    elif head == 'GAP_FC_relu_DO':
    
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(dropout)(x)
        output = keras.layers.Dense(1)(x)
    
    elif head == 'MP_relu128_DO_BN_relu64':
    
        x =  tf.keras.layers.MaxPooling2D()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(dropout)(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        output = keras.layers.Dense(1)(x)
        
    elif head == 'GAP_relu128_DO_BN_relu64':
    
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(dropout)(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        output = keras.layers.Dense(1)(x)
    
    elif head == 'GAP_relu128_DO_BN_relu64_DO':
    
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(dropout)(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(0.2)(x)
        output = keras.layers.Dense(1)(x)
        
    elif head == 'GAP_FC_relu_DO_BN':
    
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(dropout)(x)
        x = keras.layers.BatchNormalization()(x)
        output = keras.layers.Dense(1)(x)

    elif head == 'GAP_FC_relu512_DO_BN_relu128_DO_BN':
    
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(dropout)(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(dropout)(x)
        x = keras.layers.BatchNormalization()(x)
        output = keras.layers.Dense(1)(x)
        
    elif head == 'GAP_relu512_DO_BN_sigmoid':
        
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(dropout)(x)
        x = keras.layers.BatchNormalization()(x)
        output = keras.layers.Dense(1,activation='sigmoid')(x)
    
    elif head =='MP_DO_FL_relu512':
        x =  tf.keras.layers.MaxPooling2D()(x)
        x = keras.layers.Dropout(dropout)(x)
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        output = keras.layers.Dense(1)(x)
        
    uncompiled_model = tf.keras.Model(inputs=inputs, outputs=output)

    print('create_model() finished...')    
    
    return uncompiled_model 

In [11]:
def create_model_2(input_shape, model_name, fine_tune_from, head, loss, lr):

    backbone = ResNet50(weights='imagenet', 
                        input_shape=input_shape,
                        include_top=False)
    
    inputs = keras.Input(shape=input_shape)
    x = inputs
    x = tf.keras.applications.resnet50.preprocess_input(x)
    x = backbone(x, training=True)

    set_trainable = False
    for layer in backbone.layers:
        if fine_tune_from in layer.name :
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False

    if head == 'GPA_sigmoid':
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        output = keras.layers.Dense(1, activation='sigmoid')(x)
    
    elif head == 'GPA_logits':
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        output = keras.layers.Dense(1)(x)      
    
    elif head =='MP_DO_FL_relu512':
        x =  tf.keras.layers.MaxPooling2D()(x)
        x = keras.layers.Dropout(0.2)(x)
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        output = keras.layers.Dense(1)(x)
        
    model = tf.keras.Model(inputs=inputs, outputs=output)
    
    if loss == 'mse':

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                    loss=tf.keras.losses.MeanSquaredError(),
                    metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
    return model

In [12]:
def compile_model(uncompiled_model, optimizer, loss, learning_rate):
    
    print('compile_model() started...')
    
    if optimizer == 'Adam':
        optzr = Adam
    elif optimizer == 'SGD':
        optzr= SGD
        
    if loss == 'mse':
        uncompiled_model.compile(
                    optimizer=optzr(learning_rate=learning_rate),
                    loss=tf.keras.losses.MeanSquaredError(),
                    metrics=[tf.keras.metrics.RootMeanSquaredError()])
        
    elif loss == 'bce_logits':
        
        uncompiled_model.compile(
                  optimizer=optzr(learning_rate=learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
        
    elif loss == 'bce_no_logits':
        uncompiled_model.compile(
                  optimizer=optzr(learning_rate=learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
    print('compile_model() finished...')
    
    return uncompiled_model

1. add batchNormalization after DO (Dropout is meant to block information from certain neurons completely to make sure the neurons do not co-adapt. So, the batch normalization has to be after dropout otherwise you are passing information through normalization statistics. https://stackoverflow.com/questions/39691902/ordering-of-batch-normalization-and-dropout)
2. change number of outputs 128-64 
3. change flatten to GAP
4. change relu to Swish & SiLU SELU or ELU, use LeCun initialization.
4. Adam to SGD and made more epochs

In [13]:
def train_model(model, train_data, test_data, batch_size = None, epochs = None,
                steps_per_epoch = None, validation_steps = None):
    
    print('train_model() started...')
    #print('model is: ', model.summary())

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)
    
    history = model.fit(train_data, 
              validation_data = test_data,
              batch_size = batch_size, 
              epochs = epochs,
              steps_per_epoch = steps_per_epoch,
              validation_steps = validation_steps,
              verbose = 2,
              callbacks=[WandbCallback(
                monitor="val_loss", verbose=0, mode="auto", save_weights_only=(False),
                log_weights=(True), log_gradients=(True), save_model=(True),
                training_data=train_data, validation_data=None, labels=[], data_type=None,
                predictions=0, generator=None, input_type=None, output_type=None,
                log_evaluation=(False), validation_steps=None, class_colors=None,
                log_batch_frequency=None, log_best_prefix="best_", save_graph=(True),
                validation_indexes=None, validation_row_processor=None,
                prediction_row_processor=None, infer_missing_processors=(False),
                log_evaluation_frequency=0)])
    
    print('train_model() finished...')

    return history

In [14]:
def train(config=None):
    
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        print('agent received config: ', config)

        ##loader = build_dataset(config.batch_size)
        # adding our loaders 
    
        ## to do: from config add
        ##        batch size
        ##        pictures augmentation 
        ##        including augmentations functions 
        print('===================')
        print('clearing session...')
        print('===================')
        tf.keras.backend.clear_session()
        
        path = '/kaggle/input/petfinder-pawpularity-score/'
        train_gen_flow = load_train(path, config.batch_size)
        test_gen_flow = load_test(path, config.batch_size)
        
        # build model 
#         pre_trained_model = config['pre_trained_model']
#         fine_tune_from = config['tune_from']
#         head = config['head']
#         activation = config['head']
#         input_shape = config['input_shape']
        
        
        uncompiled_model = create_model(tuple(config.input_shape), 
                              config.pre_trained_model, 
                              #config.tune_from, 
                              config.head,
                              config.dropout)
        
        # compile model
        
        lr_with_decay = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=config.learning_rate,
            decay_steps=100, decay_rate=0.96,
            staircase=True)
        
        compiled_model = compile_model(uncompiled_model, config.optimizer, config.loss_function, lr_with_decay)
        
        #print('compiled model summary is: ', compiled_model.summary())
        history = train_model(compiled_model, train_gen_flow, test_gen_flow, config.batch_size, config.epochs)
        

In [15]:
wandb.login(key='620527a80f5b194ce6ba9498879a2ebe65db428d')
# 620527a80f5b194ce6ba9498879a2ebe65db428d

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [16]:
sweep_config = {
  'framework': 'TF',
  'method': 'random', 
  'note':'no_tuning',
  'metric': {
      'name': 'val_root_mean_squared_error',
      'goal': 'minimize'
  },
  'parameters': {
      'batch_size': {
          'values': [16
                    # , 64
                    ]
      },
      'learning_rate':{
          'values': [#0.01, 
                     #0.005, 
                     0.00001 #, 
                     #0.0005, 
                     #0.0001
          ]
      },
      'optimizer': {
        'values': [#'Adam', 
                   'SGD'
        ]
        },
#     'fc_layer_size': {
#         'values': [128, 256, 512]
#         },
    'dropout': {
          'values': [0.0
                    # , 0.3, 0.4, 0.5
                    ]
        },
    'input_shape': {
          'values': [(224, 224, 3)]
        },
    'pre_trained_model': {
          'values': ['ResNet50'
                     #, 'VGG16'
                    ]
        },
#     'tune_from': {
#         'values': ['conv3']
#     },
    'head': {
        'values': ['GPA_FC_sigmoid'
                    #'GPA_sigmoid' #,
                   #'MP_sigmoid'
        ]
#                    'Flatten_FC_sigmoid_DO', 
#                    'Flatten_FC_relu_DO',
#                    'GAP_FC_relu_DO',
#                    'MP_relu128_DO_BN_relu64',
#                    'GAP_relu128_DO_BN_relu64',
#                    'GAP_relu128_DO_BN_relu64_DO',
#                    'GAP_FC_relu_DO_BN',
#                    'GAP_FC_relu512_DO_BN_relu128_DO_BN',
#                    'GAP_relu512_DO_BN_sigmoid',
#                    'MP_DO_FL_relu512']
    },
    'epochs': {
        'values': [25]
    },
    'loss_function': {
        'values': [#'mse',
                   #'bce_logits', 
                   'bce_no_logits']
    }
  }
}

# Sweep config

In [17]:
sweep_config

{'framework': 'TF',
 'method': 'random',
 'note': 'no_tuning',
 'metric': {'name': 'val_root_mean_squared_error', 'goal': 'minimize'},
 'parameters': {'batch_size': {'values': [16]},
  'learning_rate': {'values': [1e-05]},
  'optimizer': {'values': ['SGD']},
  'dropout': {'values': [0.0]},
  'input_shape': {'values': [(224, 224, 3)]},
  'pre_trained_model': {'values': ['ResNet50']},
  'head': {'values': ['GPA_FC_sigmoid']},
  'epochs': {'values': [25]},
  'loss_function': {'values': ['bce_no_logits']}}}

In [18]:
sweep_id = wandb.sweep(sweep_config, project="petfinder",entity="42")

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Additional properties are not allowed ('note', 'framework' were unexpected)


Create sweep with ID: aezgjv6a
Sweep URL: https://wandb.ai/42/petfinder/sweeps/aezgjv6a


In [19]:
#sweep_id = '7dpo2ue4'

In [20]:
#wandb.agent(sweep_id, train, count=5)
wandb.agent(
    sweep_id, function=train, entity="42", project="petfinder", count=1
)

wandb: Agent Starting Run: xd83sf6a with config:
wandb: 	batch_size: 16
wandb: 	dropout: 0
wandb: 	epochs: 25
wandb: 	head: GPA_FC_sigmoid
wandb: 	input_shape: [224, 224, 3]
wandb: 	learning_rate: 1e-05
wandb: 	loss_function: bce_no_logits
wandb: 	optimizer: SGD
wandb: 	pre_trained_model: ResNet50
wandb: Currently logged in as: makcfd (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


agent received config:  {'batch_size': 16, 'dropout': 0, 'epochs': 25, 'head': 'GPA_FC_sigmoid', 'input_shape': [224, 224, 3], 'learning_rate': 1e-05, 'loss_function': 'bce_no_logits', 'optimizer': 'SGD', 'pre_trained_model': 'ResNet50'}
clearing session...
Found 7187 validated image filenames.
Found 2395 validated image filenames.
create_model() started...
loading pretrained model ResNet50...


2022-01-24 11:54:21.386571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-24 11:54:21.463713: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-24 11:54:21.464449: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-24 11:54:21.465661: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

94781440/94765736 [==============================] - 1s 0us/step
create_model() pictures mormalization 
create_model() pictures mormalization done...
create_model() finished...
compile_model() started...
compile_model() finished...
train_model() started...


2022-01-24 11:54:26.502949: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


...

2022-01-24 11:54:31.216189: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


450/450 - 160s - loss: 0.7339 - root_mean_squared_error: 0.2572 - val_loss: 0.7294 - val_root_mean_squared_error: 0.2625


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/25
450/450 - 120s - loss: 0.7207 - root_mean_squared_error: 0.2502 - val_loss: 0.7205 - val_root_mean_squared_error: 0.2575
Epoch 3/25
450/450 - 119s - loss: 0.7175 - root_mean_squared_error: 0.2495 - val_loss: 0.7177 - val_root_mean_squared_error: 0.2572
Epoch 4/25
450/450 - 119s - loss: 0.7130 - root_mean_squared_error: 0.2470 - val_loss: 0.7139 - val_root_mean_squared_error: 0.2543
Epoch 5/25
450/450 - 119s - loss: 0.7118 - root_mean_squared_error: 0.2468 - val_loss: 0.7143 - val_root_mean_squared_error: 0.2558
Epoch 6/25
450/450 - 119s - loss: 0.7096 - root_mean_squared_error: 0.2455 - val_loss: 0.7113 - val_root_mean_squared_error: 0.2534
Epoch 7/25
450/450 - 120s - loss: 0.7072 - root_mean_squared_error: 0.2439 - val_loss: 0.7104 - val_root_mean_squared_error: 0.2530
Epoch 8/25
450/450 - 119s - loss: 0.7080 - root_mean_squared_error: 0.2446 - val_loss: 0.7082 - val_root_mean_squared_error: 0.2517
Epoch 9/25
450/450 - 119s - loss: 0.7061 - root_mean_squared_error: 0.2431 -

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▅▄▃▃▃▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▂
root_mean_squared_error,█▅▅▃▃▃▂▂▂▂▂▂▁▁▂▂▁▂▁▁▂▁▁▂▂
val_loss,█▆▅▄▄▃▃▂▃▂▃▂▂▂▂▂▂▂▂▂▂▁▁▂▂
val_root_mean_squared_error,█▆▅▄▅▄▃▃▄▃▃▂▂▂▂▃▃▂▂▃▃▁▁▃▃
best_epoch,22
best_val_loss,0.70263
epoch,24
loss,0.7053
root_mean_squared_error,0.24319
val_loss,0.70698
